In [ ]:
# https://ballotpedia.org/Hawaii_Increase_Time_for_Senate_to_Act_on_Judicial_Appointments_Amendment_(2026)

ballot = """THE SENATE S.B. NO. 121 THIRTY-THIRD LEGISLATURE, 2025 STATE OF HAWAII A BILL FOR AN ACT Proposing an amendment to article vi, section 3, of the hawaii state constitution to allow the senate more time to confirm judicial appointments. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF HAWAII: SECTION 1.  The legislature finds that the Hawaii State Constitution requires the senate to act within thirty days upon receipt of a judicial appointment.  The legislature further finds that judicial appointments can, and have been, made throughout the year including during the interim when the senate is not in session, thus requiring the senate to call itself back to convene a special session to consider those appointments.  In recent years, the timing of judicial appointments has necessitated two special sessions and, in 2022, three special sessions.  Due to the thirty-day window, some special sessions have addressed just one or two appointments.  Additionally, some special sessions have occurred in mid- or late November, not far from the start of the legislative session in January.  Each special session costs approximately $5,000 for airfare and per diem for neighbor island senators, in addition to senator and staff time. The legislature concludes that lengthening the period of time allowed for the senate to consider judicial appointments made when the senate is not in regular session or is about to adjourn the regular session will provide flexibility in the scheduling of special sessions and may reduce the number of special sessions. Accordingly, the purpose of this Act is to propose an amendment to article VI, section 3, of the Hawaii State Constitution to allow the senate sixty days during certain times of the year in which to consider and act on the appointments of justices and judges. SECTION 2.  Article VI, section 3, of the Constitution of the State of Hawaii is amended to read as follows: "APPOINTMENT OF JUSTICES AND JUDGES Section 3.  The governor, with the consent of the senate, shall fill a vacancy in the office of the chief justice, supreme court, intermediate appellate court and circuit courts by appointing a person from a list of not less than four, and not more than six nominees for the vacancy presented to the governor by the judicial selection commission. If the governor fails to make any appointment within thirty days of presentation, or within ten days of the senate's rejection of any previous appointment, the appointment shall be made by the judicial selection commission from the list with the consent of the senate.  If the senate fails to reject any appointment within thirty days thereof, or sixty days if the appointment is made between April 1 and December 31, the senate shall be deemed to have consented to that appointment.  If the senate rejects any appointment, the governor shall make another appointment from the list within ten days thereof.  The same appointment and consent procedure shall be followed until a valid appointment has been made, or failing this, the judicial selection commission shall make the appointment from the list, without senate consent. The chief justice, with the consent of the senate, shall fill a vacancy in the district courts by appointing a person from a list of not less than four and not more than six nominees for the vacancy presented to the chief justice by the judicial selection commission.  If the chief justice fails to make any appointment within thirty days of presentation, or within ten days of the senate's rejection of any previous appointment, the appointment shall be made by the judicial selection commission from the list with the consent of the senate.  If the senate fails to reject any appointment within thirty days thereof, or sixty days if the appointment is made between April 1 and December 31, the senate shall be deemed to have consented to that appointment.  If the senate rejects any appointment, the chief justice shall make another appointment from the list within ten days thereof.  The same appointment and consent procedure shall be followed until a valid appointment has been made, or failing this, the judicial selection commission shall make the appointment from the list, without senate consent.  The chief justice shall appoint per diem district court judges as provided by law. The judicial selection commission shall disclose to the public the list of nominees for each vacancy concurrently with the presentation of each list to the governor or the chief justice, as applicable. QUALIFICATIONS FOR APPOINTMENT Justices and judges shall be residents and citizens of the State and of the United States, and licensed to practice law by the supreme court.  A justice of the supreme court, judge of the intermediate appellate court and judge of the circuit court shall have been so licensed for a period of not less than ten years preceding nomination.  A judge of the district court shall have been so licensed for a period of not less than five years preceding nomination. No justice or judge shall, during the term of office, engage in the practice of law, or run for or hold any other office or position of profit under the United States, the State or its political subdivisions. TENURE; RETIREMENT The term of office of justices and judges of the supreme court, intermediate appellate court and circuit courts shall be ten years.  Judges of district courts shall hold office for the periods as provided by law.  At least six months before the expiration of a justice's or judge's term of office, every justice and judge shall petition the judicial selection commission to be retained in office or shall inform the commission of an intention to retire.  If the judicial selection commission determines that the justice or judge should be retained in office, the commission shall renew the term of office of the justice or judge for the period provided by this section or by law. Justices and judges shall be retired upon attaining the age of seventy years.  They shall be included in any retirement law of the State." SECTION 3.  The question to be printed on the ballot shall be as follows: "Shall the Constitution of the State of Hawaii be amended to allow the Senate more time to consider and act on the appointments of justices and judges, by increasing the timeframe from thirty days to sixty days for appointments made between April 1 and December 31 when the senate is not in regular session or is about to adjourn the regular session?" SECTION 4.  New constitutional material is underscored. SECTION 5.  This amendment shall take effect upon compliance with article XVII, section 3, of the Constitution of the State of Hawaii. INTRODUCED BY: _____________________________ Report Title: Justices and Judges; Appointment; Confirmation; Constitutional Amendment Description: Proposes an amendment to the Hawaii State Constitution to extend the amount of time allowed the Senate to consider and act on the appointments of justices and judges from thirty to sixty days for appointments made between April 1 and December 31 when the Senate is not in Regular Session or is about to adjourn the Regular Session. The summary description of legislation appearing on this page is for informational purposes only and is not legislation or evidence of legislative intent."""





In [2]:
"""
Ballot Measure Summarization Workflow

This workflow takes a ballot measure text and:
1. Summarizes it in plain language (300 words)
2. Identifies key stakeholders using structured JSON output
3. Creates a moderated network of stakeholder agents to provide interactive perspectives
"""

import json
import random
from typing import Dict, List, Any, Tuple
from litellm import completion
from plurals.agent import Agent
from plurals.deliberation import Graph, Moderator
from dotenv import load_dotenv

load_dotenv("../src/.env")

def summarize_ballot_measure(ballot_text: str, model: str = "gpt-4o") -> str:
    """
    Step 1: Summarize the ballot measure in plain language

    Args:
        ballot_text: The full text of the ballot measure
        model: LLM model to use

    Returns:
        Plain language summary in ~300 words
    """

    prompt = f"""
    Please summarize the following ballot measure in plain, accessible language that any voter could understand.
    Focus on:
    - What the measure would actually do
    - Who would be affected
    - Key arguments for and against

    Limit your response to approximately 300 words.

    BALLOT MEASURE TEXT:
    {ballot_text}
    """

    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


def identify_stakeholders(ballot_text: str, model: str = "gpt-4o") -> Dict[str, List[str]]:
    """
    Step 2: Identify key stakeholders using structured JSON output

    Args:
        ballot_text: The full text of the ballot measure
        model: LLM model to use

    Returns:
        Dictionary with stakeholder categories and specific stakeholders (max 6 total)
    """

    prompt = f"""
    Analyze the following ballot measure and identify ONLY the most important stakeholders.

    Return EXACTLY this JSON structure with NO MORE than 3 groups in each category:
    {{
        "primary_supporters": ["group1", "group2", "group3"],
        "primary_opponents": ["group1", "group2", "group3"]
    }}

    CRITICAL: Return exactly 3 supporters and 3 opponents, no more, no less.
    Use specific, realistic stakeholder groups (e.g., "teachers unions", "property owners", "taxpayer advocacy groups").

    BALLOT MEASURE TEXT:
    {ballot_text}
    """

    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )

    stakeholders = json.loads(response.choices[0].message.content)

    # Enforce limits in case the LLM didn't follow instructions exactly
    if "primary_supporters" in stakeholders:
        stakeholders["primary_supporters"] = stakeholders["primary_supporters"][:3]
    if "primary_opponents" in stakeholders:
        stakeholders["primary_opponents"] = stakeholders["primary_opponents"][:3]

    return stakeholders


def create_stakeholder_agents(stakeholders: Dict[str, List[str]], model: str = "gpt-4o") -> List[Agent]:
    """
    Step 3: Create Agent objects for each stakeholder group

    Args:
        stakeholders: Dictionary of stakeholder categories and groups
        model: LLM model to use

    Returns:
        List of Agent objects representing different stakeholders
    """

    agents = []

    # Create agents for each stakeholder group
    for category, groups in stakeholders.items():
        for group in groups:
            persona = f"You are a representative of {group}. Think and respond from their perspective, considering their interests, concerns, and priorities."

            agent = Agent(
                persona=persona,
                model=model,
                kwargs={'temperature': 0.7}
            )

            agents.append(agent)

    return agents


def generate_random_network(num_agents: int, connection_probability: float = 0.4) -> List[Tuple[int, int]]:
    """
    Generate a random directed acyclic graph (DAG) of agents

    Args:
        num_agents: Number of agents in the network
        connection_probability: Probability that any two agents are connected

    Returns:
        List of directed edges (src_idx, dst_idx) forming a DAG
    """
    edges = []

    # Generate connections only from lower-indexed agents to higher-indexed agents
    # This guarantees no cycles (creates a DAG)
    for i in range(num_agents):
        for j in range(i + 1, num_agents):  # Only connect to higher-indexed agents
            if random.random() < connection_probability:
                edges.append((i, j))

    # Ensure connectivity by creating a "backbone" chain if needed
    if not edges or num_agents > 1:
        # Add at least one connection per agent (except the first)
        for i in range(1, num_agents):
            # Connect to at least one previous agent if not already connected
            if not any(edge[1] == i for edge in edges):
                source = random.randint(0, i-1)
                edges.append((source, i))

    return edges


def run_stakeholder_deliberation(agents: List[Agent], ballot_summary: str,
                                ballot_text: str, model: str = "gpt-4o") -> str:
    """
    Step 4: Run a moderated network deliberation where stakeholders can respond to each other

    Args:
        agents: List of stakeholder Agent objects
        ballot_summary: Plain language summary from step 1
        ballot_text: Original ballot measure text
        model: LLM model to use for moderator

    Returns:
        Moderated synthesis of stakeholder perspectives after network deliberation
    """

    task = f"""
    Based on the ballot measure summary and any previous responses you see, provide your stakeholder group's perspective.

    If you see responses from other stakeholders:
    - Acknowledge points you agree or disagree with
    - Respond to concerns raised by others
    - Build on or counter their arguments

    Address:
    - Your group's position (support/oppose) and reasoning
    - Response to other stakeholders' concerns (if any)
    - What aspects matter most to your constituency
    - Potential compromises or deal-breakers

    Keep your response to 200 words maximum.

    BALLOT MEASURE SUMMARY:
    {ballot_summary}

    FULL BALLOT MEASURE TEXT:
    {ballot_text}
    """

    # Generate a random network topology
    num_agents = len(agents)
    edges = generate_random_network(num_agents, connection_probability=0.5)

    print(f"Generated network with {len(edges)} connections between {num_agents} stakeholders")

    # Create combination instructions for interactive dialogue
    dialogue_instructions = """
    Build on the previous responses you see. If other stakeholders raised concerns:
    - Directly address their points
    - Explain where you agree or disagree
    - Provide counter-arguments or supporting evidence
    - Look for potential common ground

    Previous stakeholder responses: ${previous_responses}
    """

    # Set dialogue instructions for all agents
    for agent in agents:
        agent.combination_instructions = dialogue_instructions

    # Create a moderator to synthesize the network conversation
    moderator = Moderator(
        persona="You are a neutral policy analyst synthesizing a multi-stakeholder conversation about a ballot measure.",
        model=model,
        combination_instructions="""
        Analyze this network conversation between stakeholders to provide a comprehensive summary:

        - Map out the key arguments from supporters vs opponents
        - Highlight where stakeholders responded to each other's concerns
        - Identify areas of unexpected agreement or common ground
        - Note the most compelling arguments from each side
        - Describe how stakeholders' positions evolved through dialogue
        - Summarize any potential compromises or middle-ground solutions mentioned

        Focus on the interactive elements - how stakeholders built on, challenged, or refined each other's ideas.

        Previous stakeholder conversation: ${previous_responses}
        """
    )

    # Create and run the graph-based deliberation
    graph = Graph(
        agents=agents,
        edges=edges,
        moderator=moderator,
        task=task,
        combination_instructions=dialogue_instructions
    )

    graph.process()
    return graph.final_response


def ballot_measure_workflow(ballot_text: str, model: str = "gpt-4o") -> Dict[str, Any]:
    """
    Complete workflow for analyzing a ballot measure

    Args:
        ballot_text: The full text of the ballot measure
        model: LLM model to use throughout the workflow

    Returns:
        Dictionary containing all outputs from the workflow
    """

    print("Step 1: Summarizing ballot measure...")
    summary = summarize_ballot_measure(ballot_text, model)

    print("Step 2: Identifying stakeholders...")
    stakeholders = identify_stakeholders(ballot_text, model)

    print("Step 3: Creating stakeholder agents...")
    agents = create_stakeholder_agents(stakeholders, model)

    print(f"Step 4: Running network deliberation with {len(agents)} stakeholder agents...")
    deliberation_result = run_stakeholder_deliberation(agents, summary, ballot_text, model)

    return {
        "summary": summary,
        "stakeholders": stakeholders,
        "num_agents": len(agents),
        "stakeholder_analysis": deliberation_result
    }


# Example usage
if __name__ == "__main__":
    # Example ballot measure text

    results = ballot_measure_workflow(ballot)

    print("\n" + "="*50)
    print("BALLOT MEASURE ANALYSIS RESULTS")
    print("="*50)

    print("\n1. PLAIN LANGUAGE SUMMARY:")
    print("-" * 30)
    print(results["summary"])

    print("\n2. IDENTIFIED STAKEHOLDERS:")
    print("-" * 30)
    for category, groups in results["stakeholders"].items():
        print(f"{category.replace('_', ' ').title()}:")
        for group in groups:
            print(f"  - {group}")
        print()

    print(f"\n3. STAKEHOLDER DELIBERATION ({results['num_agents']} perspectives):")
    print("-" * 30)
    print(results["stakeholder_analysis"])

Step 1: Summarizing ballot measure...
Step 2: Identifying stakeholders...
Step 3: Creating stakeholder agents...
Step 4: Running network deliberation with 6 stakeholder agents...
Generated network with 11 connections between 6 stakeholders

BALLOT MEASURE ANALYSIS RESULTS

1. PLAIN LANGUAGE SUMMARY:
------------------------------
This ballot measure proposes a change to the Hawaii State Constitution regarding how much time the Hawaii Senate has to review and confirm judicial appointments. Currently, the Senate must act within 30 days to approve or reject a judicial appointment. This measure would extend that timeframe to 60 days for appointments made between April 1st and December 31st when the Senate is not in a regular session or is about to adjourn.

The measure primarily affects the Hawaii State Senate and individuals nominated as justices or judges in Hawaii. By extending the review period, the Senate could potentially schedule fewer special sessions, which are convened when the S